In [57]:
import numpy as np
import pandas as pd
import orca
import os; os.chdir('/home/amelia/ual_model_workspace/spring-2019-models/')
import warnings; warnings.simplefilter('ignore')
from urbansim.utils import misc
import pandana as pdna
import geopandas as gp
import warnings; warnings.simplefilter('ignore')


In [58]:
from scripts import datasources, models, variables, utils

Registering model step 'auto_ownership'
Registering model step 'TOD_choice'
Registering model step 'primary_mode_choice'
Registering model step 'WLCM'


In [59]:
orca.run(['initialize_network_small', 'initialize_network_walk','impute_missing_skims']) 

Running step 'initialize_network_small'
Time to execute step 'initialize_network_small': 0.00 s
Running step 'initialize_network_walk'
Time to execute step 'initialize_network_walk': 0.00 s
Running step 'impute_missing_skims'
Time to execute step 'impute_missing_skims': 109.49 s
Total time to execute iteration 1 with iteration value None: 109.49 s


In [3]:
chts_data = '/home/data/fall_2018/CHTS_csv_format/data'
!ls '/home/data/fall_2018/CHTS_csv_format/data'

ASSN_TravelDate.csv  Deliv_LD.csv     Deliv_VEH.csv    LookUp_PER.csv
Deliv_ACTIVITY.csv   Deliv_PER.csv    LookUp_Home.csv  LookUp_PLACE.csv
Deliv_HH.csv	     Deliv_PLACE.csv  LookUp_LD.csv


Read in cleaned data

In [71]:
school_trips_clean = pd.read_csv("school_trips_clean.csv")
school_trips_clean.head()

,Unnamed: 0,SAMPN,PERNO,RELAT,GEND,AGE,HISP,RACE1,RACE2,RACE3,...,o_priv_transit,local_rapid_bus,exp_comm_bus,school_bus,pub_shuttle,o_bus,bart_metro_r_pu,metro_bl_gr_go,street_cable_car,RF
0,0,1050385.0,4.0,3.0,1.0,10.0,2.0,1.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,1,1051388.0,3.0,3.0,1.0,10.0,2.0,1.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,2,1051388.0,3.0,3.0,1.0,10.0,2.0,1.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,4,1059276.0,2.0,3.0,1.0,15.0,1.0,1.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,6,1173970.0,3.0,3.0,1.0,8.0,1.0,1.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# for c in school_trips_clean.columns:
#     print(c)

In [22]:
hh_df = pd.read_csv('/home/data/CHTS_csv_format/data/Deliv_HH.csv')[
  ['SAMPN','HHVEH',
   'OWN','INCOM','HHSIZ']]

hh_df.head()

,SAMPN,HHVEH,OWN,INCOM,HHSIZ
0,1031985,2,1,3,2
1,1032036,1,1,7,5
2,1032053,2,2,2,6
3,1032425,2,1,7,2
4,1032558,0,2,1,1


#### 1. Merge hh dataframe with persons from cleaning notebook on SAMPN (household id) to get household-level demographic features

In [72]:
trips_hhvars = pd.merge(school_trips_clean, hh_df, on='SAMPN')
trips_hhvars.head()

,Unnamed: 0,SAMPN,PERNO,RELAT,GEND,AGE,HISP,RACE1,RACE2,RACE3,...,pub_shuttle,o_bus,bart_metro_r_pu,metro_bl_gr_go,street_cable_car,RF,HHVEH,OWN,INCOM,HHSIZ
0,0,1050385.0,4.0,3.0,1.0,10.0,2.0,1.0,NaN,NaN,...,0,0,0,0,0,0,2,1,7,4
1,1,1051388.0,3.0,3.0,1.0,10.0,2.0,1.0,NaN,NaN,...,0,0,0,0,0,0,2,1,7,3
2,2,1051388.0,3.0,3.0,1.0,10.0,2.0,1.0,NaN,NaN,...,0,0,0,0,0,0,2,1,7,3
3,4,1059276.0,2.0,3.0,1.0,15.0,1.0,1.0,NaN,NaN,...,0,0,0,0,0,0,1,2,3,2
4,6,1173970.0,3.0,3.0,1.0,8.0,1.0,1.0,NaN,NaN,...,0,0,0,0,0,0,1,2,2,4


#### 2. Categorize races, incomes and ages and create dummy variables for mn logit


In [73]:
trips_hhvars['minority'] = np.where((trips_hhvars['HISP'].isin([1.0]) |
                                trips_hhvars['RACE1'].isin([2.0]) | trips_hhvars['RACE2'].isin([2.0]) | trips_hhvars['RACE3'].isin([2.0]) | trips_hhvars['RACE4'].isin([2.0]) |
                              trips_hhvars['RACE1'].isin([3.0]) | trips_hhvars['RACE2'].isin([3.0]) | trips_hhvars['RACE3'].isin([3.0]) | trips_hhvars['RACE4'].isin([3.0]) |
                              trips_hhvars['RACE1'].isin([4.0]) | trips_hhvars['RACE2'].isin([4.0]) | trips_hhvars['RACE3'].isin([4.0]) | trips_hhvars['RACE4'].isin([4.0]) |
                              trips_hhvars['RACE1'].isin([5.0]) | trips_hhvars['RACE2'].isin([5.0]) | trips_hhvars['RACE3'].isin([5.0]) | trips_hhvars['RACE4'].isin([5.0]) |
                              trips_hhvars['RACE1'].isin([97.0]) | trips_hhvars['RACE2'].isin([97.0]) | trips_hhvars['RACE3'].isin([97.0]) | trips_hhvars['RACE4'].isin([97.0])),1,0)

trips_hhvars['HISP'] = np.where(trips_hhvars['HISP'].isin([1.0]),1,0)
trips_hhvars['black'] = np.where((trips_hhvars['RACE1'].isin([2.0]) | trips_hhvars['RACE2'].isin([2.0]) | trips_hhvars['RACE3'].isin([2.0]) | trips_hhvars['RACE4'].isin([2.0])),1,0)
trips_hhvars['native'] = np.where((trips_hhvars['RACE1'].isin([3.0]) | trips_hhvars['RACE2'].isin([3.0]) | trips_hhvars['RACE3'].isin([3.0]) | trips_hhvars['RACE4'].isin([3.0])),1,0)
trips_hhvars['asian'] = np.where((trips_hhvars['RACE1'].isin([4.0]) | trips_hhvars['RACE2'].isin([4.0]) | trips_hhvars['RACE3'].isin([4.0]) | trips_hhvars['RACE4'].isin([4.0])),1,0)
trips_hhvars['PI'] = np.where((trips_hhvars['RACE1'].isin([5.0]) | trips_hhvars['RACE2'].isin([5.0]) | trips_hhvars['RACE3'].isin([5.0]) | trips_hhvars['RACE4'].isin([5.0])),1,0)


In [74]:
trips_hhvars['hh_inc_less10k'] = np.where(trips_hhvars['INCOM'].isin([1.0]),1,0)
trips_hhvars['hh_inc_10kless25k'] = np.where(trips_hhvars['INCOM'].isin([2.0]),1,0)
trips_hhvars['hh_inc_25kless35k'] = np.where(trips_hhvars['INCOM'].isin([3.0]),1,0)
trips_hhvars['hh_inc_35kless50k'] = np.where(trips_hhvars['INCOM'].isin([4.0]),1,0)
trips_hhvars['hh_inc_50kless75k'] = np.where(trips_hhvars['INCOM'].isin([5.0]),1,0)
#trips_hhvars trips1['hh_inc_75kless100k'] = np.where(trips1['INCOM'].isin([6.0]),1,0)
trips_hhvars['hh_inc_100kless150k'] = np.where(trips_hhvars['INCOM'].isin([7.0]),1,0)
trips_hhvars['hh_inc_150kless200k'] = np.where(trips_hhvars['INCOM'].isin([8.0]),1,0)
trips_hhvars['hh_inc_200kless250k'] = np.where(trips_hhvars['INCOM'].isin([9.0]),1,0)
# trips1['250kplus'] = np.where(trips1['INCOM'].isin([10.0]),1,0)

trips_hhvars['hh_inc_less50k'] = np.where(trips_hhvars['INCOM'].isin([1.0,2.0,3.0,4.0]),1,0)
# trips1['less75k'] = np.where(trips1['INCOM'].isin([1.0,2.0,3.0,4.0,5.0]),1,0)
# trips1['150kless250k'] = np.where(trips1['INCOM'].isin([8.0,9.0]),1,0)
# trips1['150kplus'] = np.where(trips1['INCOM'].isin([8.0,9.0,10.0]),1,0)

trips_hhvars['hh_inc_less75k'] = np.where(trips_hhvars['INCOM'].isin([1.0,2.0,3.0,4.0,5.0]),1,0)
trips_hhvars['hh_inc_75kless100k'] = np.where(trips_hhvars['INCOM'].isin([6.0]),1,0)
trips_hhvars['hh_inc_150kplus'] = np.where(trips_hhvars['INCOM'].isin([8.0,9.0,10.0]),1,0)
trips_hhvars['hh_inc_150kless250k'] = np.where(trips_hhvars['INCOM'].isin([8.0,9.0]),1,0)
trips_hhvars['hh_inc_250kplus'] = np.where(trips_hhvars['INCOM'].isin([10.0]),1,0)

In [69]:
# trips_hhvars['lessGED'] = np.where(trips_hhvars['EDUCA'].isin([1.0]),1,0)
# trips_hhvars['GED'] = np.where(trips_hhvars['EDUCA'].isin([2.0]),1,0)
# trips_hhvars['somebach'] = np.where(trips_hhvars['EDUCA'].isin([3.0]),1,0)
# trips_hhvars['Assoc'] = np.where(trips_hhvars['EDUCA'].isin([4.0]),1,0)
# trips_hhvars['Bach'] = np.where(trips_hhvars['EDUCA'].isin([5.0]),1,0)

# trips_hhvars['lessGED_GED'] = np.where(trips_hhvars['EDUCA'].isin([1.0,2.0]),1,0)

# trips_hhvars['lessbach'] = (trips_hhvars['EDUCA'] < 5).astype(int)


In [75]:
trips_hhvars['age_2less5'] = (trips_hhvars["AGE"] >= 2.0) & (trips_hhvars["AGE"] < 5.0)#pre k/k
trips_hhvars['age_5less10'] = (trips_hhvars["AGE"] >= 5.0) & (trips_hhvars["AGE"] < 10.0)  #1st-5th (elem)
trips_hhvars['age_10less14'] = (trips_hhvars["AGE"] >= 10.0) & (trips_hhvars["AGE"] < 14.0) #6th-8th (middle)
trips_hhvars['age_14less19'] = (trips_hhvars["AGE"] >= 14.0) & (trips_hhvars["AGE"] < 19.0)#9th-12th (high)



In [76]:
trips_hhvars['female'] = trips_hhvars['GEND'] - 1

trips_hhvars['tenure_2'] = trips_hhvars['OWN'] - 1

trips_hhvars['noveh'] = np.where(trips_hhvars.HHVEH.isin([0.0]),1,0)
trips_hhvars['1veh'] = np.where(trips_hhvars.HHVEH.isin([1.0]),1,0)
trips_hhvars['2veh'] = np.where(trips_hhvars.HHVEH.isin([2.0]),1,0)
trips_hhvars['3plusveh'] = np.where(trips_hhvars.HHVEH.between(3,8,inclusive = True),1,0)

trips_hhvars['hh_size_1per'] = np.where(trips_hhvars.HHSIZ.isin([1.0]),1,0)
trips_hhvars['hh_size_2per'] = np.where(trips_hhvars.HHSIZ.isin([2.0]),1,0)
trips_hhvars['hh_size_3per'] = np.where(trips_hhvars.HHSIZ.isin([3.0]),1,0)
trips_hhvars['hh_size_4plusper'] = np.where(trips_hhvars.HHSIZ.between(4,8,inclusive = True),1,0)

#### 3. Add BEAM skims for generalized cost

In [77]:
beam_skims = orca.get_table('beam_skims').to_frame()
reset_beam_skims = beam_skims.reset_index()


In [78]:
reset_beam_skims.head()

,from_zone_id,to_zone_id,dist,gen_cost_BIKE,gen_cost_CAR,gen_cost_DRIVE_TRANSIT,gen_cost_RIDE_HAIL,gen_cost_RIDE_HAIL_POOLED,gen_cost_RIDE_HAIL_TRANSIT,gen_cost_WALK,gen_cost_WALK_TRANSIT,gen_tt_BIKE,gen_tt_CAR,gen_tt_DRIVE_TRANSIT,gen_tt_RIDE_HAIL,gen_tt_RIDE_HAIL_POOLED,gen_tt_RIDE_HAIL_TRANSIT,gen_tt_WALK,gen_tt_WALK_TRANSIT
0,1,1,193.1208,0.248434,0.114126,0.145693,0.576147,0.316090,0.309961,28.510923,3.505719,0.805269,0.173001,0.286490,0.310197,0.241026,0.273580,3.395910,1.850000
1,1,2,386.2416,0.496869,0.228251,0.291386,1.152294,0.632179,0.619923,57.021846,22.054913,1.610537,0.346003,0.572980,0.620393,0.482052,0.547160,6.791820,23.037028
2,1,3,708.1096,0.910926,0.418461,0.534207,2.112539,1.158995,1.136525,104.540052,40.434006,2.952651,0.634338,1.050464,1.137388,0.883762,1.003127,12.451669,42.234551
3,1,4,659.8294,0.848817,0.389929,0.497784,1.968503,1.079973,1.059035,97.412321,37.677142,2.751334,0.591088,0.978841,1.059839,0.823505,0.934732,11.602692,39.354922
4,1,5,1094.3512,1.407795,0.646712,0.825593,3.264834,1.791175,1.756448,161.561898,62.488919,4.563188,0.980341,1.623444,1.757781,1.365814,1.550287,19.243489,65.271579


In [81]:
trips_skims = pd.merge(reset_beam_skims, trips_hhvars,  how='right', right_on=['zone_id_home','school_zone_id'], left_on = ['from_zone_id','to_zone_id'])
trips_skims.head()



,from_zone_id,to_zone_id,dist,gen_cost_BIKE,gen_cost_CAR,gen_cost_DRIVE_TRANSIT,gen_cost_RIDE_HAIL,gen_cost_RIDE_HAIL_POOLED,gen_cost_RIDE_HAIL_TRANSIT,gen_cost_WALK,...,female,tenure_2,noveh,1veh,2veh,3plusveh,hh_size_1per,hh_size_2per,hh_size_3per,hh_size_4plusper
0,12.0,99.0,8529.5020,10.972519,5.040548,6.434768,25.446497,13.960627,13.689966,1259.232441,...,0.0,0,0,1,0,0,0,0,1,0
1,17.0,20.0,2414.0100,3.105430,1.426570,1.821161,13.701745,3.951121,3.874519,7.865417,...,0.0,0,0,1,0,0,0,0,0,1
2,17.0,20.0,2414.0100,3.105430,1.426570,1.821161,13.701745,3.951121,3.874519,7.865417,...,0.0,0,0,1,0,0,0,0,0,1
3,29.0,808.0,43709.6744,56.228982,25.830430,32.975153,130.401294,71.541627,70.154619,6452.972284,...,0.0,0,0,0,1,0,0,1,0,0
4,36.0,40.0,1416.2192,1.821852,3.609543,1.068414,4.225079,2.317991,2.273051,209.080103,...,0.0,0,0,1,0,0,0,0,0,1


In [83]:
for c in trips_skims.columns:
    print(c)

from_zone_id
to_zone_id
dist
gen_cost_BIKE
gen_cost_CAR
gen_cost_DRIVE_TRANSIT
gen_cost_RIDE_HAIL
gen_cost_RIDE_HAIL_POOLED
gen_cost_RIDE_HAIL_TRANSIT
gen_cost_WALK
gen_cost_WALK_TRANSIT
gen_tt_BIKE
gen_tt_CAR
gen_tt_DRIVE_TRANSIT
gen_tt_RIDE_HAIL
gen_tt_RIDE_HAIL_POOLED
gen_tt_RIDE_HAIL_TRANSIT
gen_tt_WALK
gen_tt_WALK_TRANSIT
Unnamed: 0
SAMPN
PERNO
RELAT
GEND
AGE
HISP
RACE1
RACE2
RACE3
RACE4
O_RACE
NTVTY
STUDE
SCITY_persons
SSTAT
SZIP_persons
EDUCA
SNAME_lookup
SCITY_lookup
SZIP_lookup
SCTFIP_lookup
STRACT_lookup
SBLOCK
SPrimaryCity_lookup
SSTFIP_lookup
HCITY
HZIP
HXCORD
HYCORD
HBLOCK
HTRACT
HCTFIP
HPrimaryCity
parcel_id_home
parcel_id_work
zone_id_home
zone_id_work
school_id
HHPER
HHPERPLA
APURP1
APURP2
APURP3
PNAME
trip_type
ARR_HR
ARR_MIN
DEP_HR
DEP_MIN
TRIPDUR
CDSCode
School
District
County
Street
City
Zip
State
Latitude
Longitude
type
grade_0
grade_1
grade_2
grade_3
grade_4
grade_5
grade_6
grade_7
grade_8
grade_9
grade_10
grade_11
grade_12
Kindergarten Enrollment
Grade 1 Enrollme

#### 5. Use random forest as a first pass at feature importances

In [82]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
rf = RandomForestClassifier(n_estimators=100, random_state=0)

In [84]:
X = trips_skims[['from_zone_id', 'to_zone_id', 'dist', 'gen_cost_BIKE', 'gen_cost_CAR', 'gen_cost_DRIVE_TRANSIT', 
        'gen_cost_RIDE_HAIL', 'gen_cost_RIDE_HAIL_POOLED', 'gen_cost_RIDE_HAIL_TRANSIT', 'gen_cost_WALK',
'gen_cost_WALK_TRANSIT', 'gen_tt_BIKE', 'gen_tt_CAR', 'gen_tt_DRIVE_TRANSIT', 'gen_tt_RIDE_HAIL',
'gen_tt_RIDE_HAIL_POOLED', 'gen_tt_RIDE_HAIL_TRANSIT', 'gen_tt_WALK', 'gen_tt_WALK_TRANSIT']]
X
# y = trips2['modeHW']

# rf.fit(X0,y) 
# importances = rf.feature_importances_
# indices = np.argsort(importances)[::-1]

,from_zone_id,to_zone_id,dist,gen_cost_BIKE,gen_cost_CAR,gen_cost_DRIVE_TRANSIT,gen_cost_RIDE_HAIL,gen_cost_RIDE_HAIL_POOLED,gen_cost_RIDE_HAIL_TRANSIT,gen_cost_WALK,gen_cost_WALK_TRANSIT,gen_tt_BIKE,gen_tt_CAR,gen_tt_DRIVE_TRANSIT,gen_tt_RIDE_HAIL,gen_tt_RIDE_HAIL_POOLED,gen_tt_RIDE_HAIL_TRANSIT,gen_tt_WALK,gen_tt_WALK_TRANSIT
0,12.0,99.0,8529.5020,10.972519,5.040548,6.434768,25.446497,13.960627,13.689966,1259.232441,487.045986,35.566028,7.640892,12.653314,13.700353,10.645313,12.083116,149.986018,508.734363
1,17.0,20.0,2414.0100,3.105430,1.426570,1.821161,13.701745,3.951121,3.874519,7.865417,9.079937,10.065857,2.162517,3.581127,9.966667,3.012824,3.419750,24.366667,11.316667
2,17.0,20.0,2414.0100,3.105430,1.426570,1.821161,13.701745,3.951121,3.874519,7.865417,9.079937,10.065857,2.162517,3.581127,9.966667,3.012824,3.419750,24.366667,11.316667
3,29.0,808.0,43709.6744,56.228982,25.830430,32.975153,130.401294,71.541627,70.154619,6452.972284,2495.880942,182.259117,39.155967,64.842267,70.207844,54.552206,61.920269,768.607596,2607.023641
4,36.0,40.0,1416.2192,1.821852,3.609543,1.068414,4.225079,2.317991,2.273051,209.080103,80.868013,5.905303,6.266667,2.100928,2.274776,1.767524,2.006253,24.903339,84.469102
5,36.0,40.0,1416.2192,1.821852,3.609543,1.068414,4.225079,2.317991,2.273051,209.080103,80.868013,5.905303,6.266667,2.100928,2.274776,1.767524,2.006253,24.903339,84.469102
6,38.0,39.0,1094.3512,1.407795,0.646712,0.825593,7.560726,1.791175,1.756448,3.462514,62.488919,4.563188,0.980341,1.623444,4.433333,1.365814,1.550287,11.395833,65.271579
7,38.0,39.0,1094.3512,1.407795,0.646712,0.825593,7.560726,1.791175,1.756448,3.462514,62.488919,4.563188,0.980341,1.623444,4.433333,1.365814,1.550287,11.395833,65.271579
8,39.0,39.0,321.8680,0.414057,0.190209,0.242821,0.960245,0.526816,0.516602,47.518205,3.898976,1.342114,0.288336,0.477484,0.516994,0.401710,0.455967,5.659850,3.733333
9,39.0,66.0,5600.5032,7.204597,3.309643,4.225093,16.708266,9.166600,8.988883,826.816773,319.796233,23.352788,5.017039,8.308214,8.995703,6.989753,7.933819,98.481386,334.036902
